In [ ]:
! pip install transformers datasets evaluate
! pip install torch torchvision
! pip install tokenizers
! pip3 install emoji==0.6.0
! pip install -U accelerate
! pip install -U transformers
! pip install pandas
! pip install numpy
! pip install scikit-learn
! pip install openai

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv("bragging_data.csv", engine = "python")

from sklearn.model_selection import train_test_split

df['label'].replace(['not', 'achievement', 'action', 'feeling', 'trait', 'possession', 'affiliation'], [0, 1,1,1,1,1,1], inplace = True)
df.drop(labels = ["id", "sampling", "round"], axis = 1, inplace = True)
train, test = train_test_split(df, test_size=0.5, random_state = 5)

promptarr = df['text'].to_list()



In [ ]:
import time
import os
import openai


openai.api_key = os.getenv('API_KEY')
print(openai.api_key)

In [ ]:
texts = []
answers = []

few_shot_training = train.groupby("label").head(2)
few_shot_training
# bragging: 6686, 5317
# not bragging: 6615, 3380


In [ ]:
for i in range (1, 200):
    
    messages = [({"role": "system", "content": "You classify whether or not tweets contain bragging"})]
    messages.append({"role": "system", "content": "This tweet contains bragging:" + few_shot_training['text'][6686]})
    messages.append({"role": "system", "content": "This tweet contains bragging:" + few_shot_training['text'][5317]})
    messages.append({"role": "system", "content": "This tweet does not contain bragging:" + few_shot_training['text'][6615]})
    messages.append({"role": "system", "content": "This tweet does not contain bragging:" + few_shot_training['text'][3380]})

    content = "Does this tweet contain bragging? respond with '1' if yes or '0' if no. tweet: [" + promptarr[i] + "]"
    print(content)
    messages.append({"role": "user", "content": content})
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages = messages
    )

    
    response = completion
    texts.append(content)
    answers.append(completion.choices[0].message.content)
    print(answers)
    time.sleep(30)


In [ ]:
#print(texts)
#print(answers)

textarr = np.array(texts)
answersarr = np.array(answers)

dataset = pd.DataFrame({'label': answersarr, 'text': textarr}, columns = ['label', 'text'])
dataset
dataset.to_csv('GPT3fewshotGuess.csv')

In [ ]:
#gpt3

guesses3 = pd.read_csv("GPT3zeroshotGuess.csv", engine = "python")

labelpred = guesses3['label']

real = df['label'].head(200)

mask = labelpred == real

numcorrect = sum(mask)

zeroshot = numcorrect/200

#gpt3 few shot

guesses4 = pd.read_csv("GPT3fewshotGuess.csv", engine = "python")

labelpred2 = guesses4['label']

real2 = df['label'].head(163)

mask2 = labelpred2 == real2

numcorrect2 = sum(mask2)

fewshot = numcorrect2/163

print(zeroshot)
print(fewshot)

#labelpred2